# Worksheet 05

Name:  Sai Tejaswini Junnuri
UID: U92492167

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

1. Assign the points to the initial centroids.
    Let 0 be centroid for Cluster 1 and 2 for Cluster 2
    The clusters assigned are [1,1,2,2,2,2,2]
Continue till the centroids converge
2. Calculate the new centroids for each cluster
    For Cluster 1 , the data points are [0, 0.5] - the new centroid is 0.25
    For Cluster 2 , the data points are [1.5, 2, 6, 6.5, 7] - the new centroid is 4.6
3. Assign the points to the new centroids
    The clusters assigned are [1,1,1,1,2,2,2]
2. For Cluster 1 , the data points are [0, 0.5, 1.5, 2] - the new centroid is 1
   For Cluster 2 , the data points are [6, 6.5, 7] - the new centroid is 6.5
3. Assign the points to the new centroids
    The clusters assigned are [1,1,1,1,2,2,2] which are the same as before
    
Hence, the final clusters and respective points are Cluster 1 - [0, 0.5, 1.5, 2] and Cluster 2 - [6, 6.5, 7]

b) Describe in plain english what the cost function for k means is.

It is the sum of eucledian distance between the centroid of each cluster to all the points in that cluster for all the clusters

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

K-means starts with an initial set of cluster centroids. Depending on how these initial centroids are chosen, the algorithm can converge to different local optima

d) Does Lloyd's Algorithm always converge? Why / why not?

Lloyd's algorithm will always converge as it is greedy and will keep reducing the cost function till local optima is reached

e) Follow along in class the implementation of Kmeans

In [18]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))
    
    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), self.k, replace = False)]
    
    def distance(self,a,b):
        return ((a[0]-b[0])**2+(a[1]-b[1])**2)**(0.5)
    
    def get_centers(self):
        centers = []
        for i in set(self.assignment):
            
            cluster = []
            for j in range(len(self.data)):
                if self.assignment[j] ==i:
                    cluster.append(self.data[j])
            x = 0
            y = 0
            for delta in range(len(cluster)):
                x+=cluster[delta][0]
                y+=cluster[delta][1]
            centers.append([x/len(cluster), y/len(cluster)])
        
        return np.array(centers)
        
        
    def assign(self, centers):
        for i in range(len(self.data)):
            closest_center = 0
            closest_distance = self.distance(self.data[i], centers[closest_center])
            for j in range(1,len(centers)):
                if self.distance(self.data[i],centers[j]) < closest_distance:
                    closest_center = j
                    closest_distance = self.distance(self.data[i],centers[j])
            self.assignment[i] = closest_center

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = centers[::-1]
        flag = 0
        while np.array_equiv(centers,new_centers) != True:
            if flag==1:
                centers = new_centers
            new_centers = self.get_centers()
            self.assign(new_centers)
            self.snap(new_centers)
            flag = 1
        
        return
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)